# Dog Breed Classification Project Proposal

### Project Background and Goal
The project will focus on using deep learning to identify dog breed from a supplied dog image. In specific, when given an arbitary image, the algorithm should first identify if it is a dog or a human. If dog is detected, the code will return the estimate of breed. If human is detected, the code will return the resembling dog breed.

### Data Set

Two data set are provide by Udacity.
1. Dog data set (https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip):
    * 8351 dog images of 133 dog breed categories
    * split into train, validation and test set, with 6680, 835, 836 images respectively.
2. Human data set (http://vis-www.cs.umass.edu/lfw/lfw.tgz):
    * 13233 human face images

### Proposed Solution

For detecting human or dog, we can use either image classification or object detection for this problem. As this is a pretty standard task, there are some existing implementations we can use directly. For example, we can use OpenCV's face detector for human detection.

For dog breed classification, we can either train a nerual network from scratch or use transfer learning. The architecture nerual network of image classification typically consists of two part: feature extraction part and classifier part. The feature extraction part usually consists of a series of convolutional layers. The classifier usually consists of a few fully-connected/linear layers and the final layer would out put N_class numbers that can be used to compute loss function. For transfer learning, we can use a pretrained model, e.g. VGG16, ResNet50, change the last layer to match the number of classis in our problem. We will fix the weight of feature extraction part, and fine tune the classifier part.

### Benchmark Model

For dog breed classification, I tried to use AWS Sagemaker's build in image classification. I choose to use pretrained 18 layer network and fine tune the last layer. However, I can only archieve 1% accuracy on both training and validation, and early stopping was triggered before 30 epochs. The algorithm is kind of black-box to me and I am not sure how to improve it.

Below are my code to start a image classification training job on Sagemaker:

In [ ]:
ic = sagemaker.estimator.Estimator(
                training_image,
                role, 
                train_instance_count=1, 
                train_instance_type='ml.p2.xlarge',
                train_volume_size = 50,
                train_max_run = 360000,
                input_mode= 'File',
                output_path=s3_output_location,
                sagemaker_session=sess,
                base_job_name='project-dog',
                    )
ic.set_hyperparameters(num_layers=18,
                             use_pretrained_model=1,
                             image_shape = "3,224,224",
                             num_classes=133,
                             num_training_samples=6678,
                             mini_batch_size=32,
                             epochs=30,
                             learning_rate=0.001,
                             precision_dtype='float32',
                             early_stopping=True,
                             optimizer='adam',
                             checkpoint_frequency=5,
                    )

### Evaluation Metrics

For dog/human detection, we can evaluate the accuracy. We also would like to focus on false positive and false negative rate. For example, given a set of human images, the human detector can failed to detect how many images. Given a set of dog images, how many images the human detector may mistakenly detect dog as human.

For dog breed classification, as this is a multiclass classification, we can use cross-entropy as optimization and evaluation metric. We can also use accuracy as evaluation metric.